In [1]:
import numpy as np
import pandas as pd
table = 'prevApp'


In [2]:
app_train = pd.read_csv('../1_data_import/application_train.csv')
prev_app = pd.read_csv('../1_data_import/previous_application.csv')

In [10]:
# get rid of system errors

prev_app = prev_app[prev_app.FLAG_LAST_APPL_PER_CONTRACT != 'N']
prev_app = prev_app[prev_app.NFLAG_LAST_APPL_IN_DAY != 0]

# get rid of columns helping identified system errors
columns = ['FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY']
prev_app = prev_app.drop(columns, axis=1)


AttributeError: 'DataFrame' object has no attribute 'FLAG_LAST_APPL_PER_CONTRACT'

In [11]:
prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#one-hot encoding
#NAME_CONTRACT_TYPE
#NAME_CLIENT_TYPE bd wiedziec czy nowy czy stary
#NAME_CONTRACT_STATUS
#NAME_SELLER_INDUSTRY b.duzo unikalnych wartosci

In [7]:
rangeFunc = lambda x: x.max() - x.min()
PREVIOUS_APPLICATION_AGGREGATION_RECIPIES = []
for agg in ['mean', 'min', 'max', 'sum', 'var', 'std']:
    for select in ['AMT_ANNUITY',
                   'AMT_APPLICATION',
                   'AMT_CREDIT',
                   'AMT_DOWN_PAYMENT',
                   'AMT_GOODS_PRICE',
                   'CNT_PAYMENT',
                   'DAYS_DECISION',
                   'HOUR_APPR_PROCESS_START',
                   'RATE_DOWN_PAYMENT',
                   'DAYS_TERMINATION',
                   'DAYS_LAST_DUE',
                   'DAYS_LAST_DUE_1ST_VERSION',
                   'DAYS_FIRST_DUE',
                   'DAYS_FIRST_DRAWING'
                   ]:
        PREVIOUS_APPLICATION_AGGREGATION_RECIPIES.append((select, agg))
PREVIOUS_APPLICATION_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], PREVIOUS_APPLICATION_AGGREGATION_RECIPIES)]

In [8]:
application = app_train[['SK_ID_CURR']]
groupby_aggregate_names = []
groupby_cols = PREVIOUS_APPLICATION_AGGREGATION_RECIPIES[0][0]
specs = PREVIOUS_APPLICATION_AGGREGATION_RECIPIES[0][1]
group_object = prev_app.groupby(groupby_cols)
for select, agg in specs:
    groupby_aggregate_name = table + '_' + agg + '_' + select
    application = application.merge(group_object[select]
                          .agg(agg)
                          .reset_index()
                          .rename(index=str,
                                  columns={select: groupby_aggregate_name})
                          [groupby_cols + [groupby_aggregate_name]],
                          on = groupby_cols,
                          how = 'left')
    groupby_aggregate_names.append(groupby_aggregate_name)

#application.head()

,SK_ID_CURR,prevApp_mean_AMT_ANNUITY,prevApp_mean_AMT_APPLICATION,prevApp_mean_AMT_CREDIT,prevApp_mean_AMT_DOWN_PAYMENT,prevApp_mean_AMT_GOODS_PRICE,prevApp_mean_CNT_PAYMENT,prevApp_mean_DAYS_DECISION,prevApp_mean_HOUR_APPR_PROCESS_START,prevApp_mean_RATE_DOWN_PAYMENT,...,prevApp_var_RATE_DOWN_PAYMENT,prevApp_std_AMT_ANNUITY,prevApp_std_AMT_APPLICATION,prevApp_std_AMT_CREDIT,prevApp_std_AMT_DOWN_PAYMENT,prevApp_std_AMT_GOODS_PRICE,prevApp_std_CNT_PAYMENT,prevApp_std_DAYS_DECISION,prevApp_std_HOUR_APPR_PROCESS_START,prevApp_std_RATE_DOWN_PAYMENT
0,100002,9251.775,179055.00,179055.00,0.00,179055.00,24.000000,-606.000000,9.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,56553.990,435436.50,484191.00,3442.50,435436.50,10.000000,-1305.000000,14.666667,0.050030,...,0.005006,46332.557777,424161.620549,497949.861808,4868.430188,424161.620549,3.464102,898.138631,2.516611,0.070754
2,100004,5357.250,24282.00,20106.00,4860.00,24282.00,4.000000,-815.000000,5.000000,0.212008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,23651.175,272203.26,291695.50,34840.17,408304.89,23.000000,-272.444444,14.666667,0.163412,...,0.005923,13623.580119,286175.129541,333337.354853,45462.482973,253670.602059,20.228692,157.236375,1.000000,0.076958
4,100007,12278.805,150530.25,166638.75,3390.75,150530.25,20.666667,-1222.833333,12.333333,0.159516,...,0.007050,8063.586466,100585.988276,118032.409509,404.111525,100585.988276,14.348054,717.268546,2.503331,0.083967


In [9]:
# applying range function
PREVIOUS_APPLICATION_AGGREGATION_RANGE = []
for col in ['AMT_ANNUITY',
            'AMT_APPLICATION',
            'AMT_CREDIT',
            'AMT_DOWN_PAYMENT',
            'AMT_GOODS_PRICE',
            'CNT_PAYMENT',
            'DAYS_DECISION',
            'HOUR_APPR_PROCESS_START',
            'RATE_DOWN_PAYMENT',
            'DAYS_TERMINATION',
            'DAYS_LAST_DUE',
            'DAYS_LAST_DUE_1ST_VERSION',
            'DAYS_FIRST_DUE',
            'DAYS_FIRST_DRAWING'
           ]:
        PREVIOUS_APPLICATION_AGGREGATION_RANGE.append((col, 'rangeFunc'))

groupby_aggregate_names = []

group_object = prev_app.groupby(groupby_cols)
for select, agg in PREVIOUS_APPLICATION_AGGREGATION_RANGE:
    groupby_aggregate_name = table + '_' + agg + '_' + select
    application = application.merge(group_object[select]
                          .agg(rangeFunc)
                          .reset_index()
                          .rename(index=str,
                                  columns={select: groupby_aggregate_name})
                          [groupby_cols + [groupby_aggregate_name]],
                          on = groupby_cols,
                          how = 'left')
    groupby_aggregate_names.append(groupby_aggregate_name)


,SK_ID_CURR,prevApp_mean_AMT_ANNUITY,prevApp_mean_AMT_APPLICATION,prevApp_mean_AMT_CREDIT,prevApp_mean_AMT_DOWN_PAYMENT,prevApp_mean_AMT_GOODS_PRICE,prevApp_mean_CNT_PAYMENT,prevApp_mean_DAYS_DECISION,prevApp_mean_HOUR_APPR_PROCESS_START,prevApp_mean_RATE_DOWN_PAYMENT,...,prevApp_std_RATE_DOWN_PAYMENT,prevApp_rangeFunc_AMT_ANNUITY,prevApp_rangeFunc_AMT_APPLICATION,prevApp_rangeFunc_AMT_CREDIT,prevApp_rangeFunc_AMT_DOWN_PAYMENT,prevApp_rangeFunc_AMT_GOODS_PRICE,prevApp_rangeFunc_CNT_PAYMENT,prevApp_rangeFunc_DAYS_DECISION,prevApp_rangeFunc_HOUR_APPR_PROCESS_START,prevApp_rangeFunc_RATE_DOWN_PAYMENT
0,100002,9251.775,179055.00,179055.00,0.00,179055.00,24.000000,-606.000000,9.000000,0.000000,...,NaN,0.000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.000000
1,100003,56553.990,435436.50,484191.00,3442.50,435436.50,10.000000,-1305.000000,14.666667,0.050030,...,0.070754,91619.685,831190.5,967828.5,6885.00,831190.50,6.0,1595.0,5.0,0.100061
2,100004,5357.250,24282.00,20106.00,4860.00,24282.00,4.000000,-815.000000,5.000000,0.212008,...,NaN,0.000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.000000
3,100006,23651.175,272203.26,291695.50,34840.17,408304.89,23.000000,-272.444444,14.666667,0.163412,...,0.076958,37471.590,688500.0,906615.0,64293.66,661587.66,48.0,436.0,3.0,0.108836
4,100007,12278.805,150530.25,166638.75,3390.75,150530.25,20.666667,-1222.833333,12.333333,0.159516,...,0.083967,20844.495,230323.5,269784.0,571.50,230323.50,38.0,1983.0,7.0,0.118747


In [40]:
# add feature representing cnt number of prev app per SK_ID_CURR
df_toMerge = group_object['SK_ID_PREV'].nunique().reset_index().rename(index=str,columns={select: groupby_aggregate_name})
cols = ['SK_ID_CURR', 'Cnt_prevApp']
df_toMerge.columns = cols
application = application.merge(df_toMerge,
                          on = groupby_cols,
                          how = 'left')


In [43]:
#get number of different values for given series
def diff_vals(x):
    dv = x.nunique()
    return dv
#get most frequent value for given series
def most_freq_val(x):
    if x.shape[0] == 0:
        res = np.nan
    else:
        try:
            res = x.value_counts().idxmax()
        except ValueError:
            res = np.nan
    return res

In [70]:
PREVIOUS_APPLICATION_AGGREGATION_cntUNIQUE = []
for col in ['NAME_CONTRACT_TYPE',
            'WEEKDAY_APPR_PROCESS_START',
            'NAME_CASH_LOAN_PURPOSE',
            'NAME_PAYMENT_TYPE',
            'CODE_REJECT_REASON',
            'NAME_TYPE_SUITE',
            'NAME_PAYMENT_TYPE',
            'NAME_CASH_LOAN_PURPOSE',
            'SK_ID_PREV'
           ]:
        PREVIOUS_APPLICATION_AGGREGATION_cntUNIQUE.append((col, 'cntU'))
groupby_aggregate_names = []

for select, agg in PREVIOUS_APPLICATION_AGGREGATION_RANGE:
    groupby_aggregate_name = table + '_' + agg + '_' + select
    application = application.merge(group_object[select]
                          .agg(diff_vals)
                          .reset_index()
                          .rename(index = str,
                                  columns = {select: groupby_aggregate_name})
                          [groupby_cols + [groupby_aggregate_name]],
                          on = groupby_cols,
                          how = 'left')
    groupby_aggregate_names.append(groupby_aggregate_name)
application.head()

,SK_ID_CURR,prevApp_mean_AMT_ANNUITY,prevApp_mean_AMT_APPLICATION,prevApp_mean_AMT_CREDIT,prevApp_mean_AMT_DOWN_PAYMENT,prevApp_mean_AMT_GOODS_PRICE,prevApp_mean_CNT_PAYMENT,prevApp_mean_DAYS_DECISION,prevApp_mean_HOUR_APPR_PROCESS_START,prevApp_mean_RATE_DOWN_PAYMENT,...,Cnt_prevApp_y,prevApp_rangeFunc_AMT_ANNUITY_y,prevApp_rangeFunc_AMT_APPLICATION_y,prevApp_rangeFunc_AMT_CREDIT_y,prevApp_rangeFunc_AMT_DOWN_PAYMENT_y,prevApp_rangeFunc_AMT_GOODS_PRICE_y,prevApp_rangeFunc_CNT_PAYMENT_y,prevApp_rangeFunc_DAYS_DECISION_y,prevApp_rangeFunc_HOUR_APPR_PROCESS_START_y,prevApp_rangeFunc_RATE_DOWN_PAYMENT_y
0,100002,9251.775,179055.00,179055.00,0.00,179055.00,24.000000,-606.000000,9.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,100003,56553.990,435436.50,484191.00,3442.50,435436.50,10.000000,-1305.000000,14.666667,0.050030,...,3.0,3.0,3.0,3.0,2.0,3.0,2.0,3.0,3.0,2.0
2,100004,5357.250,24282.00,20106.00,4860.00,24282.00,4.000000,-815.000000,5.000000,0.212008,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,100006,23651.175,272203.26,291695.50,34840.17,408304.89,23.000000,-272.444444,14.666667,0.163412,...,9.0,6.0,7.0,7.0,2.0,6.0,4.0,4.0,2.0,2.0
4,100007,12278.805,150530.25,166638.75,3390.75,150530.25,20.666667,-1222.833333,12.333333,0.159516,...,6.0,6.0,6.0,6.0,2.0,6.0,5.0,6.0,5.0,2.0


In [66]:
#features representing number of unique categorical values (where applicable) eg. for how many different products client(s) applied
#prev_app.groupby('NAME_CASH_LOAN_PURPOSE').SK_ID_PREV.nunique()

In [14]:
prev_app[prev_app.SK_ID_CURR == 456244].sort_values('SK_ID_PREV')

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
1667875,1035930,456244,Cash loans,21177.810,180000.0,191880.0,0.0,180000.0,THURSDAY,13,...,XNA,12.0,high,Cash Street: high,365243.0,-2119.0,-1789.0,-1879.0,-1874.0,0.0
671166,1061673,456244,Cash loans,62820.000,1125000.0,1125000.0,NaN,1125000.0,MONDAY,12,...,XNA,24.0,low_normal,Cash X-Sell: low,365243.0,-933.0,-243.0,-393.0,-383.0,0.0
101886,1159964,456244,Cash loans,14425.515,121500.0,129519.0,NaN,121500.0,TUESDAY,10,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-1429.0,-1099.0,-1189.0,-1181.0,1.0
1505372,1165146,456244,Cash loans,44312.535,900000.0,1042767.0,NaN,900000.0,FRIDAY,8,...,XNA,36.0,low_normal,Cash X-Sell: low,NaN,NaN,NaN,NaN,NaN,NaN
1557018,1623269,456244,Cash loans,9906.390,135000.0,152820.0,0.0,135000.0,FRIDAY,9,...,XNA,24.0,middle,Cash X-Sell: middle,365243.0,-1552.0,-862.0,-1372.0,-1363.0,1.0
882795,1842078,456244,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,12,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
723585,1842417,456244,Cash loans,NaN,0.0,0.0,NaN,NaN,WEDNESDAY,11,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
1472002,1842431,456244,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,12,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
260493,1936634,456244,Cash loans,36824.625,315000.0,332464.5,0.0,315000.0,THURSDAY,8,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-1791.0,-1461.0,-1551.0,-1545.0,1.0
1640426,1967722,456244,Cash loans,17232.030,135000.0,162778.5,0.0,135000.0,TUESDAY,8,...,XNA,12.0,middle,Cash X-Sell: middle,NaN,NaN,NaN,NaN,NaN,NaN
